In [ ]:
import pandas as pd
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple
import os
from datetime import datetime
import pdb;
from grocery_ml import GroceryML
from hidden_layer_param_builder import HiddenLayerParamSetBuilder

pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.float_format", lambda x: f"{x:.6f}")
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 2000)

print(os.getcwd())
# print("GPUs Available:", tf.config.list_physical_devices('GPU'))
#tf.debugging.set_log_device_placement(True)

def run_all_experiments(df, model_param_sets, output_dir):
    total = len(model_param_sets)
    print(f"run_all_experiments() when: {datetime.now()}  output_dir: {output_dir}");
    for index, params in enumerate(model_param_sets, 1):
        print(f"Running Exp {index}/{total}...")
        groceryML.run_experiment(df,  params["buildParams"], params["trainParams"], output_dir)


try:
    groceryML = GroceryML();
    groceryMLCore = GroceryMLCore();
    groceryML.build_training_df()
    if groceryML.training_df is None:
        raise();
    ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    groceryML.training_df.to_csv(f"training_df-{ts}.csv");
except Exception as ex: 
    print(ex)
    ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    groceryML.training_df.to_csv(f"training_df-{ts}-exception.csv");


In [ ]:

# quick test shot
layers = [
    { "units": 1, "activation": "relu" }
]
modelParamsList = []
modelParamsList.append({
    "trainParams": { "epochs": 1 },
    "buildParams": {
    "embedding_dim": 1,
    "layers": layers,
    "activation": "relu",
    "output_activation": "sigmoid",
    "optimizer": "adam",
    "learning_rate": 0.01,
    "loss": "binary_crossentropy",
    "metrics": "BinaryCrossentropy"
    }
})
ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
run_all_experiments(groceryML.training_df, modelParamsList, f"exp/keras/keras-1/test-shot-no-trust-{ts}")

In [ ]:
###########################################################

modelParamsList = []
layers = [
    { "units": 24, "activation": "relu" },
    { "units": 8, "activation": "relu" },
]
output_activations = ["sigmoid", "linear", "tanh"]
embDims = [16,24,48]
metrics = ["Accuracy", "MAE", "MSE", "MAPE", "MSLE", "Precision", "Recall", "AUC",  "BinaryCrossentropy", "RootMeanSquaredError"]

for act in output_activations:
    for dim in embDims:
        modelParamsList.append({
            "trainParams": { "epochs": 20 },
            "buildParams": {
                "embedding_dim": dim,
                "layers": layers,
                "output_activation": act,
                "optimizer": "adam",
                "learning_rate": 0.0001,
                "loss": "binary_crossentropy" if act == "sigmoid" else "mse",
                "metrics": metrics
            }
        })

ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
run_all_experiments( groceryML.training_df, modelParamsList, f"exp/keras/keras-1/{ts}")